In [3]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from datetime import datetime
from pathlib import Path
from matplotlib.patches import Rectangle

from utils import *

BASE_DIR = Path(os.path.abspath(''))
os.chdir(BASE_DIR)
print(f'Working directory: {os.getcwd()}')

Working directory: c:\Users\jespe\Documents\GitHub_local\ARFusion


## Functionality to perform isolate-level analysis

In [4]:
results_dict = pd.read_pickle(os.path.join(BASE_DIR, 'results', 'MM', '15_ab', 'FT_noPT_0.75_0.7', 'CV_results.pkl'))

In [5]:
results_dict.keys()

dict_keys(['train_losses', 'losses', 'accs', 'iso_accs', 'sensitivities', 'specificities', 'F1_scores', 'iso_stats', 'ab_stats'])

In [6]:
results_dict['iso_stats']

[      year  country  num_ab  num_genotypes  num_point_mutations  \
 0     2020      USA    12.0              9                    9   
 1     2020      USA    12.0             16                    9   
 2     2017      USA     6.0              1                    1   
 3     2018      USA     6.0              1                    0   
 4     2018      USA     6.0              3                    2   
 ...    ...      ...     ...            ...                  ...   
 1287  2020      USA    12.0              6                    6   
 1288  2020      USA    11.0             10                    4   
 1289  2019      USA     6.0              2                    2   
 1290  2013  Germany     3.0              2                    2   
 1291  2020      USA    12.0              3                    3   
 
                                 ab_classes  num_masked  num_masked_S  \
 0     [0, 4, 2, 2, 2, 3, 1, 0, 1, 3, 1, 0]           6             5   
 1     [0, 4, 2, 2, 2, 3, 1, 0, 1, 3